In [1]:
import pandas as pd
import os

In [2]:
os.listdir()

['.gitignore', 'inputs', 'models', 'notebooks', 'reading_tracks.ipynb', 'src']

In [3]:
# create an empty dataframe
src = pd.DataFrame()

In [4]:
src.shape

(0, 0)

In [5]:
for tracks in os.listdir("./inputs/JSON/"):
    if tracks.endswith(".json"):
        print(tracks)
        tracks = os.path.join("./inputs/JSON/", tracks)
        src = pd.concat([src, pd.read_json(tracks)], axis=0)

track1.json
track2.json
track3.json


In [6]:
colors = pd.get_dummies(src['color'])
src = pd.concat([src, colors], axis=1)
src.drop('color', axis=1, inplace=True)

In [7]:
src

,x,y,blue,orange_big,yellow
0,-60.939023,4.612072,0,1,0
1,-57.657950,5.865915,1,0,0
2,-54.528351,7.486922,1,0,0
3,-51.420276,9.949794,1,0,0
4,-49.295484,13.150159,1,0,0
...,...,...,...,...,...
165,-5.838843,12.595252,0,0,1
166,-3.246721,10.148054,0,0,1
167,-0.687852,7.665161,0,0,1
168,1.833232,5.143008,0,0,1


In [8]:
import torch

In [9]:
# convert the dataframe to a tensor
src = torch.tensor(src.values, dtype=torch.float32)

In [10]:
src

tensor([[-60.9390,   4.6121,   0.0000,   1.0000,   0.0000],
        [-57.6580,   5.8659,   1.0000,   0.0000,   0.0000],
        [-54.5284,   7.4869,   1.0000,   0.0000,   0.0000],
        ...,
        [ -0.6879,   7.6652,   0.0000,   0.0000,   1.0000],
        [  1.8332,   5.1430,   0.0000,   0.0000,   1.0000],
        [  4.3120,   2.5780,   0.0000,   0.0000,   1.0000]])